In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')
train_X = train.iloc[:,1:]
train_y = train['label']

In [ ]:
fig=plt.figure(figsize=(30,6))
for i in range(5):
    rand = random.randint(0, len(train_X)-1)
    plt.subplot(1,5,i+1)
    img = train_X.loc[rand].values
    label = train_y.loc[rand]
    img = img.reshape(28,28)
    plt.imshow(img)
    plt.title(label)

In [ ]:
fig=plt.figure(figsize=(30,6))
for i in range(5):
    rand = random.randint(0, len(test)-1)
    plt.subplot(1,5,i+1)
    img = test.loc[rand].values
    img = img.reshape(28,28)
    plt.imshow(img)
    plt.title('index '+str(rand))

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(train_X, train_y)
print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

In [ ]:
train_X = train_X.values
test_X = test_X.values
train_y = train_y.values
test_y = test_y.values
train_X = train_X.reshape(len(train_X), 28, 28)
test_X = test_X.reshape(len(test_X), 28, 28)
print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

In [ ]:
train_y_class = tf.keras.utils.to_categorical(train_y, num_classes=len(train.label.unique()))
test_y_class = tf.keras.utils.to_categorical(test_y, num_classes=len(train.label.unique()))
print(train_X.shape, test_X.shape, train_y_class.shape, test_y_class.shape)

In [ ]:
print(train_y[11])
print(train_y_class[11])

In [ ]:
'''
from imblearn.over_sampling import SMOTE
smote = SMOTE('minority')

train_X_squeeze=train_X.reshape(len(train_X),len(train_X[0]) * len(train_X[0][0]))
print(train_X_squeeze.shape)

train_X_os, train_y_class_os = smote.fit_sample(train_X_squeeze, train_y_class)
print(train_X_os.shape, train_y_class_os.shape)

for i in range(len(train_y_class_os[0])-1):
    train_X_os, train_y_class_os = smote.fit_sample(train_X_os, train_y_class_os)

print(train_X_os.shape, train_y_class_os.shape)

train_X_os = train_X_os.reshape(len(train_X_os), 28, 28)

train_X_os.shape
'''

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),input_shape=train_X[0].shape))
model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2, 2))
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2, 2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.LayerNormalization())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

#opt = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9)
#opt = RMSprop(lr=0.01)
opt = Adam(lr=0.01)
#ls = tf.keras.losses.Huber()

model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=["accuracy"])

model.summary()

In [ ]:
history = model.fit(train_X, train_y_class, epochs=30,batch_size=72, validation_data=(test_X, test_y_class),verbose=2,shuffle=False)

In [ ]:
model=tf.keras.models.load_model("model_digit_recognizer")

In [ ]:
model.summary()

In [ ]:
history=model.evaluate(test_X, test_y_class)

In [ ]:
plt.plot(history.history['accuracy'][:], label='train')
plt.plot(history.history['val_accuracy'][:], label='test')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'][:], label='train')
plt.plot(history.history['val_loss'][:], label='test')
plt.legend()
plt.show()

In [ ]:
sample_submission

In [ ]:
test_values = test.values
test_values = test_values.reshape(len(test_values), 28, 28)
test_values.shape

In [ ]:
predictions = model.predict(test_values)

In [ ]:
plt.imshow(test_values[4])

In [ ]:
np.argmax(predictions[4])

In [ ]:
aws = [int(np.argmax(x)) for x in predictions]
sample_submission['Label']=aws
sample_submission

In [ ]:
sample_submission.to_csv('submission.csv', index=False)